# Using Custom Metric Function In Binary Classification

In this notebook, we will show an example of how to calculate custom performance metrics on an H2O model for binary classification. The notebook will go through the following steps:

1. Train a GBM model in H2O
2. Write a script to calculate cost matrix based loss
3. Train a GBM model in H2O using this loss function as a [`custom_metric_func`](https://github.com/h2oai/h2o-3/blob/master/h2o-docs/src/dev/custom_functions.md)
4. Train a Grid of GBMs and choose model based on this loss function


## 1. Train a  GBM Model in H2O

In [1]:
# Load H2O library
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_181"; Java(TM) SE Runtime Environment (build 1.8.0_181-b13); Java HotSpot(TM) 64-Bit Server VM (build 25.181-b13, mixed mode)
  Starting server from /anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/wk/m00ydfj52f9fl7zvx5cjztgc0000gn/T/tmpib3zvce7
  JVM stdout: /var/folders/wk/m00ydfj52f9fl7zvx5cjztgc0000gn/T/tmpib3zvce7/h2o_patrickaboyoun_started_from_python.out
  JVM stderr: /var/folders/wk/m00ydfj52f9fl7zvx5cjztgc0000gn/T/tmpib3zvce7/h2o_patrickaboyoun_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.7
H2O cluster version age:,9 days
H2O cluster name:,H2O_from_python_patrickaboyoun_rxle0j
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [2]:
# Import Data
train_path = "https://raw.githubusercontent.com/h2oai/app-consumer-loan/master/data/loan.csv"
train = h2o.import_file(train_path, destination_frame = "loan_train")
train["bad_loan"] = train["bad_loan"].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [3]:
# Set target and predictor variables
y = "bad_loan"
x = train.col_names
x.remove(y)
x.remove("int_rate")

In [4]:
# Train GBM Model
from h2o.estimators import H2OGradientBoostingEstimator

gbm_v1 = H2OGradientBoostingEstimator(model_id = "gbm_v1.hex")

gbm_v1.train(y = y, x = x, training_frame = train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [5]:
print(gbm_v1)

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  gbm_v1.hex


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.1363951465191071
RMSE: 0.3693171354257843
LogLoss: 0.43467809200204266
Mean Per-Class Error: 0.3508577460272526
AUC: 0.7079429892082825
Gini: 0.41588597841656494
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.19860658480443358: 


,0,1,Error,Rate
0,95113.0,38858.0,0.29,(38858.0/133971.0)
1,12401.0,17615.0,0.4131,(12401.0/30016.0)
Total,107514.0,56473.0,0.3126,(51259.0/163987.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1986066,0.4073350,228.0
max f2,0.1265125,0.5633597,311.0
max f0point5,0.2803267,0.3837915,152.0
max accuracy,0.4292597,0.8203272,62.0
max precision,0.7770519,1.0,0.0
max recall,0.0438524,1.0,396.0
max specificity,0.7770519,1.0,0.0
max absolute_mcc,0.2251476,0.2462816,202.0
max min_per_class_accuracy,0.1823683,0.6488206,245.0
max mean_per_class_accuracy,0.1920415,0.6491423,235.0


Gains/Lift Table: Avg response rate: 18.30 %, avg score: 18.31 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100008,0.4806708,3.5211761,3.5211761,0.6445122,0.5303616,0.6445122,0.5303616,0.0352146,0.0352146,252.1176084,252.1176084
,2,0.0200016,0.4375856,2.8082795,3.1647278,0.5140244,0.4573837,0.5792683,0.4938727,0.0280850,0.0632996,180.8279507,216.4727796
,3,0.0300024,0.4110619,2.6750278,3.0014945,0.4896341,0.4234982,0.5493902,0.4704145,0.0267524,0.0900520,167.5027810,200.1494467
,4,0.0400032,0.3902679,2.5117945,2.8790695,0.4597561,0.4001474,0.5269817,0.4528477,0.0251199,0.1151719,151.1794482,187.9069471
,5,0.0500040,0.3731066,2.3019231,2.7636402,0.4213415,0.3814565,0.5058537,0.4385695,0.0230211,0.1381930,130.1923060,176.3640189
,6,0.1000018,0.3147513,2.0763146,2.4199984,0.3800463,0.3412588,0.4429538,0.3899171,0.1038113,0.2420043,107.6314643,141.9998372
,7,0.1499997,0.2770929,1.7224882,2.1875044,0.3152824,0.2946252,0.4003984,0.3581544,0.0861207,0.328125,72.2488239,118.7504446
,8,0.2000037,0.2503682,1.5570461,2.0298802,0.285,0.2631637,0.3715470,0.3344053,0.0778585,0.4059835,55.7046075,102.9880242
,9,0.2999994,0.2116632,1.2990293,1.7862732,0.2377729,0.2298703,0.3269575,0.2995617,0.1298974,0.5358809,29.9029331,78.6273176
,10,0.4000012,0.1833189,1.0747405,1.6083873,0.1967193,0.1968137,0.2943974,0.2738743,0.1074760,0.6433569,7.4740466,60.8387287



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
,2018-09-10 13:51:02,0.022 sec,0.0,0.3866984,0.4759704,0.5,1.0,0.8169611
,2018-09-10 13:51:02,0.764 sec,1.0,0.3847635,0.4710759,0.6582899,2.5923891,0.3376304
,2018-09-10 13:51:03,1.026 sec,2.0,0.3831611,0.4671624,0.6641427,2.7218454,0.3575466
,2018-09-10 13:51:03,1.278 sec,3.0,0.3818189,0.4639557,0.6658226,2.8447858,0.3506497
,2018-09-10 13:51:03,1.387 sec,4.0,0.3806812,0.4612690,0.6685973,2.9280752,0.3523572
---,---,---,---,---,---,---,---,---
,2018-09-10 13:51:05,3.608 sec,23.0,0.3725708,0.4423727,0.6931526,3.2979795,0.3413929
,2018-09-10 13:51:05,3.731 sec,24.0,0.3724149,0.4419880,0.6938720,3.2979795,0.3356547
,2018-09-10 13:51:06,3.887 sec,25.0,0.3722080,0.4415065,0.6947887,3.3079734,0.3273674
,2018-09-10 13:51:06,4.015 sec,26.0,0.3720656,0.4411389,0.6955963,3.3312924,0.3317702



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
term,2747.9863281,1.0,0.2493995
annual_inc,1938.3043213,0.7053544,0.1759151
addr_state,1546.3793945,0.5627318,0.1403451
revol_util,1427.8056641,0.5195825,0.1295836
purpose,934.7630005,0.3401629,0.0848365
dti,847.0342407,0.3082382,0.0768744
loan_amnt,627.7856445,0.2284530,0.0569761
emp_length,249.1807861,0.0906776,0.0226149
home_ownership,239.7884827,0.0872597,0.0217625
total_acc,163.4083862,0.0594648,0.0148305


## 2. Write Script to Calculate Cost Matrix Loss

### Function to Calculate Cost Matrix Loss in H2O

In [6]:
def CostMatrixLoss(actual, predicted, cost_tp, cost_tn, cost_fp, cost_fn):
    c1 = cost_tp + cost_tn - cost_fp - cost_fn
    c2 = cost_fn - cost_tn
    c3 = cost_fp - cost_tn
    c4 = cost_tn

    cost = (actual * predicted * c1) + (actual * c2) + (predicted * c3) + c4
    mean_cost = cost.mean()[0]
    return mean_cost

In [7]:
loss_v1 = CostMatrixLoss(train[y].asnumeric(), gbm_v1.predict(train)["p1"], 0, 0, 1, 3)
print("CostMatrixLoss: " + str(round(loss_v1, 4)))

gbm prediction progress: |████████████████████████████████████████████████| 100%
CostMatrixLoss: 0.5543


### Python Script to calculate Cost Matrix Loss in custom_metric_func

The confusion matrix loss metric is defined in a class stored in utils_model_metrics.py. This class contains three methods `map`, `reduce`, and `metric`. The `map` method takes 5 arguments `predicted`, `actual`, `weight`, `offset` and `model`.

```
class CostMatrixLossMetric:
    def map(self, predicted, actual, weight, offset, model):
        cost_tp = 0
        cost_tn = 0
        cost_fp = 1
        cost_fn = 3
        c1 = cost_tp + cost_tn - cost_fp - cost_fn
        c2 = cost_fn - cost_tn
        c3 = cost_fp - cost_tn
        c4 = cost_tn
        y = actual[0]
        p = predicted[2] # [class, p0, p1]
        return [weight * ((y * p * c1) + (y * c2) + (p * c3) + c4), weight]

    def reduce(self, left, right):
        return [left[0] + right[0], left[1] + right[1]]

    def metric(self, last):
        return last[0] / last[1]
```

This class definition is uploaded to the H2O cluster using [`h2o.upload_custom_metric`](http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/h2o.html?highlight=custom_metric#h2o.upload_custom_metric).

In [8]:
from utils_model_metrics import CostMatrixLossMetric

cost_matrix_loss_func = h2o.upload_custom_metric(CostMatrixLossMetric,
                                                 func_name = "CostMatrixLoss",
                                                 func_file = "cost_matrix_loss.py")

In [9]:
type(cost_matrix_loss_func)

str

In [10]:
print(cost_matrix_loss_func)

python:CostMatrixLoss=cost_matrix_loss.CostMatrixLossMetricWrapper


## 3. Train a GBM Model using custom_metric_func

The [`H2OGeneralizedLinearEstimator`](http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/modeling.html?highlight=automl#h2ogeneralizedlinearestimator),
[`H2ORandomForestEstimator`](http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/modeling.html?highlight=automl#h2orandomforestestimator), and
[`H2OGradientBoostingEstimator`](http://docs.h2o.ai/h2o/latest-stable/h2o-py/docs/modeling.html?highlight=automl#h2ogradientboostingestimator) models accept a `custom_metric_func` argument.

In [11]:
# Train GBM Model with custom_metric_function
gbm_v2 = H2OGradientBoostingEstimator(model_id = "gbm_v2.hex",
                                      custom_metric_func = cost_matrix_loss_func)

gbm_v2.train(y = y, x = x, training_frame = train)

gbm Model Build progress: |███████████████████████████████████████████████| 100%


In [12]:
perf = gbm_v2.model_performance()
perf


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.1363951465191071
RMSE: 0.3693171354257843
LogLoss: 0.43467809200204266
Mean Per-Class Error: 0.3508577460272526
AUC: 0.7079429892082825
Gini: 0.41588597841656494
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.19860658480443358: 


,0,1,Error,Rate
0,95113.0,38858.0,0.29,(38858.0/133971.0)
1,12401.0,17615.0,0.4131,(12401.0/30016.0)
Total,107514.0,56473.0,0.3126,(51259.0/163987.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.1986066,0.4073350,228.0
max f2,0.1265125,0.5633597,311.0
max f0point5,0.2803267,0.3837915,152.0
max accuracy,0.4292597,0.8203272,62.0
max precision,0.7770519,1.0,0.0
max recall,0.0438524,1.0,396.0
max specificity,0.7770519,1.0,0.0
max absolute_mcc,0.2251476,0.2462816,202.0
max min_per_class_accuracy,0.1823683,0.6488206,245.0
max mean_per_class_accuracy,0.1920415,0.6491423,235.0


Gains/Lift Table: Avg response rate: 18.30 %, avg score: 18.31 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100008,0.4806708,3.5211761,3.5211761,0.6445122,0.5303616,0.6445122,0.5303616,0.0352146,0.0352146,252.1176084,252.1176084
,2,0.0200016,0.4375856,2.8082795,3.1647278,0.5140244,0.4573837,0.5792683,0.4938727,0.0280850,0.0632996,180.8279507,216.4727796
,3,0.0300024,0.4110619,2.6750278,3.0014945,0.4896341,0.4234982,0.5493902,0.4704145,0.0267524,0.0900520,167.5027810,200.1494467
,4,0.0400032,0.3902679,2.5117945,2.8790695,0.4597561,0.4001474,0.5269817,0.4528477,0.0251199,0.1151719,151.1794482,187.9069471
,5,0.0500040,0.3731066,2.3019231,2.7636402,0.4213415,0.3814565,0.5058537,0.4385695,0.0230211,0.1381930,130.1923060,176.3640189
,6,0.1000018,0.3147513,2.0763146,2.4199984,0.3800463,0.3412588,0.4429538,0.3899171,0.1038113,0.2420043,107.6314643,141.9998372
,7,0.1499997,0.2770929,1.7224882,2.1875044,0.3152824,0.2946252,0.4003984,0.3581544,0.0861207,0.328125,72.2488239,118.7504446
,8,0.2000037,0.2503682,1.5570461,2.0298802,0.285,0.2631637,0.3715470,0.3344053,0.0778585,0.4059835,55.7046075,102.9880242
,9,0.2999994,0.2116632,1.2990293,1.7862732,0.2377729,0.2298703,0.3269575,0.2995617,0.1298974,0.5358809,29.9029331,78.6273176
,10,0.4000012,0.1833189,1.0747405,1.6083873,0.1967193,0.1968137,0.2943974,0.2738743,0.1074760,0.6433569,7.4740466,60.8387287



CostMatrixLoss: 0.5543038365540239


In [13]:
perf.custom_metric_name()

'CostMatrixLoss'

In [14]:
perf.custom_metric_value()

0.5543038365540239

We can see that our custom cost matrix loss function is in the model performance metrics labeled `CostMatrixLoss`.  This value matches the value calculated in our original GBM model.

In [15]:
print("Cost Matrix Loss V1: " + str(round(loss_v1, 4)))
print("Cost Matrix Loss V2: " + str(round(gbm_v2.model_performance().custom_metric_value(), 4)))

Cost Matrix Loss V1: 0.5543
Cost Matrix Loss V2: 0.5543


## 4. Train a Grid of GBMs and choose model based on custom loss metric

In [16]:
from h2o.grid.grid_search import H2OGridSearch
gbm_hyper_parameters = {'max_depth': [4, 5, 6]}
gbm_grid = H2OGridSearch(H2OGradientBoostingEstimator(custom_metric_func = cost_matrix_loss_func,
                                                      nfolds = 5),
                           gbm_hyper_parameters)
gbm_grid.train(x = x, y = y, training_frame = train, grid_id = "gbm_grid")

gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [17]:
sorted([[h2o.get_model(x).model_performance(xval = True).custom_metric_value(), x] for x in gbm_grid.model_ids])

[[0.5569432815873339, 'gbm_grid_model_2'],
 [0.5617845741639093, 'gbm_grid_model_0'],
 [0.5645126062055483, 'gbm_grid_model_1']]

## Shutdown H2O Cluster

In [18]:
h2o.cluster().shutdown()

H2O session _sid_a53c closed.
